This is the under-dev notebook for checking identity metadata of PacBio flowcells.

The order of execution is the following:

0. Load all flowcells that is workable
1. Based on the flowcells' marked identity metadata, see if its Mercury FP VCF has been uploaded to the cloud database (and upload using on-prem scripts if missing)
2. Check which flowcells have negative LOD, report
3. Check which flowcells have indecisive LOD, report
4. Check which flowcells don't have LOD yet, then
    * if it's ready, i.e. have alignments and FP vcf, then launch `VerifyFingerprint`
    * if it's because there's no BAM yet, i.e. `PBFlowcell` hasn't been run on it, launch the job
    * if it's because there's a shallow BAM, report

There are several dark-knowledge dependencies that are needed for this to run:
    * "known_samples_without_mercury.txt": holding samples that are known to have no GT'ed fingerprinting VCF
    * "known_flowcells_with_issues.txt": holding flowcells that are known to be inappropriate (for various reasons) to be fingerprinted


In [185]:
# auto reloading of local scripts under dev
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [186]:
# relying on these stdlib anyway
import http
import re
import os
import sys
import pandas as pd

In [187]:
# Google Cloud and FISS
from firecloud import api as fapi

from google.cloud import storage
storage_client = storage.Client()

In [188]:
# load local lib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.table_utils import *
from src.gcs_utils import *

In [189]:
import datetime
import dateutil
from dateutil import parser

# Date & time, for record keeping

In [190]:
today = datetime.datetime.today().strftime("%Y%m%d")
cutoff_date = pd.to_datetime(datetime.datetime(2021, 1, 1, 0, 0, tzinfo=datetime.timezone.utc))

In [191]:
print("Current Time =", datetime.datetime.now().strftime("%D %H:%M:%S"))

Current Time = 12/27/21 08:57:01


In [192]:
output_dir = f'/Users/shuang/Desktop/DailyFingerprintCheck/{datetime.datetime.today().strftime("%Y-%m-%d")}'
os.makedirs(output_dir, exist_ok=True)

# Filters to apply, still under development, so changes with time

In [193]:
known_flowcells_inappropriate_for_current_pbflowcell = ['DA143934', 'DA073901']

In [194]:
ff = GcsPath('gs://broad-gp-pacbio/metrics/fingerprinting/mercury/known_samples_without_mercury.txt')
if not ff.exists(storage_client):
    raise RuntimeError("Dependency file gs://broad-gp-pacbio/metrics/fingerprinting/mercury/known_samples_without_mercury.txt doesn't exist any more.")
known_samples_without_mercury = ff.get_blob(storage_client).download_as_text().split('\n')
print(f"{len(known_samples_without_mercury)} samples are known to have no Mercury entries.")

45 samples are known to have no Mercury entries.


In [195]:
ff = GcsPath('gs://broad-gp-pacbio/metrics/fingerprinting/mercury/known_flowcells_with_issues.txt')
if not ff.exists(storage_client):
    raise RuntimeError("Dependency file gs://broad-gp-pacbio/metrics/fingerprinting/mercury/known_flowcells_with_issues.txt doesn't exist any more.")
known_problematic_flowcells = ff.get_blob(storage_client).download_as_text().split('\n')
print(f"{len(known_problematic_flowcells)} flowcells known to have issues preventing them from being VerifyFingerprint'ed.")

23 flowcells known to have issues preventing them from being VerifyFingerprint'ed.


In [196]:
ff = GcsPath('gs://broad-gp-pacbio/metrics/fingerprinting/mercury/flowcells_identity_manually_confirmed.txt')
if not ff.exists(storage_client):
    raise RuntimeError("Dependency file gs://broad-gp-pacbio/metrics/fingerprinting/mercury/flowcells_identity_manually_confirmed.txt doesn't exist any more.")
borderline_lod_flowcells_cleared = ff.get_blob(storage_client).download_as_text().split('\n')

In [197]:
samples_in_cloud_mercury = list()
for b in storage_client.list_blobs('broad-gp-pacbio', prefix='metrics/fingerprinting/mercury/vcfs'):
    file_name = b.name.split('/')[-1]
    if file_name.endswith('.vcf.gz'):
        sample = file_name.split('__')[0]
        samples_in_cloud_mercury.append(sample)
print(f"{len(samples_in_cloud_mercury)} samples already in living in the lrma-cloud-mercury.")

287 samples already in living in the lrma-cloud-mercury.


# LOAD & FORMAT FLOWCELLS

In [198]:
primary_namespace = 'production-long-reads'
primary_workspace = 'broad-gp-pacbio'
root_data_type='sample'
flowcell_table = \
  fetch_existing_root_table(ns=primary_namespace,
                            ws=primary_workspace,
                            etype=root_data_type)

In [199]:
gcs_locations = ['aligned_bai', 'aligned_bam', 'aligned_pbi',
                 'ccs_bam', 'ccs_pbi', 'ccs_report',
                 'fingerprint_details', 'fingerprint_metrics',
                 'fq', 'gcs_input_dir', 'input_bam', 'input_pbi', 'subreads_bam', 'subreads_pbi']

In [200]:
lab_identity = ['bio_sample', 'description', 'well_sample']
sequencer_identity = ['flowcell_id', 'movie_name', 'well_name']
terra_identity = ['sample']

In [201]:
categorical_columns = {'type': 'category',
                       'columns': ['application', 'experiment_type', 'instrument', 'workspace']}

date_time_columns = {'type': 'datetime64',
                     'timezone': datetime.timezone.utc,
                     'columns': ['created_at']}

boolean_columns = {'type': 'bool',
                   'columns': ['is_ccs', 'is_corrected', 'is_isoseq']}

int_type_columns = {'type': 'Int64',
                    'columns': ['aligned_num_bases','aligned_num_reads','aligned_read_length_N50',
                                'ccs_zmws_fail_filters','ccs_zmws_input','ccs_zmws_pass_filters', 'ccs_zmws_shortcut_filters',
                                'insert_size',
                                'num_bases','num_reads','num_reads_Q10','num_reads_Q12','num_reads_Q15','num_reads_Q5','num_reads_Q7','num_records',
                                'total_length']}

float_type_columns = {'type': 'float64',
                      'columns': ['lod_expected_sample',
                                  'aligned_est_fold_cov', 'raw_est_fold_cov',
                                  'aligned_frac_bases','aligned_read_length_mean','aligned_read_length_median','aligned_read_length_stdev',
                                  'average_identity', 'median_identity',
                                  'ccs_zmws_fail_filters_pct','ccs_zmws_pass_filters_pct','ccs_zmws_shortcut_filters_pct',
                                  'polymerase_read_length_N50', 'polymerase_read_length_mean',
                                  'read_length_N50', 'read_length_mean', 'read_length_median', 'read_length_stdev', 'read_qual_mean', 'read_qual_median',
                                  'subread_read_length_N50','subread_read_length_mean']}

string_type_columns = {'type': 'str',
                       'columns': gcs_locations + terra_identity + lab_identity + sequencer_identity}

In [202]:
for n in boolean_columns['columns']:
    flowcell_table[n] = flowcell_table[n].apply(lambda s: s=='TRUE' or s=='True' or s=='true').astype(boolean_columns['type'])

In [203]:
for n in categorical_columns['columns']:
    flowcell_table[n] = flowcell_table[n].astype(categorical_columns['type'])

In [204]:
for n in string_type_columns['columns']:
    flowcell_table[n] = flowcell_table[n].astype(string_type_columns['type'])

In [205]:
def convert_to_float(e) -> float or None:
    if e:
        if e.lower() in ['nan', 'none']:
            return None
        else:
            try:
                return float(e)
            except TypeError:
                print(e)
                raise
    else:
        return None

def convert_to_int(e) -> int:
    f = convert_to_float(e)
    return round(f) if f else None

In [206]:
for n in int_type_columns['columns']:
    try:
        flowcell_table[n] = flowcell_table[n].apply(convert_to_int).astype(int_type_columns['type'])
    except ValueError:
        print(n)
        raise

In [207]:
for n in float_type_columns['columns']:
    try:
        flowcell_table[n] = flowcell_table[n].apply(convert_to_float).astype(float_type_columns['type'])
    except ValueError:
        print(n)
        raise

In [208]:
def convert_date_time(s):
    try:
        t = parser.isoparse(s).astimezone(tz=date_time_columns['timezone'])
        return pd.to_datetime(t)
    except (ValueError, pd.errors.OutOfBoundsDatetime):
        return pd.Timestamp.min
for n in date_time_columns['columns']:
    flowcell_table[n] = flowcell_table[n].apply(lambda s: pd.to_datetime(convert_date_time(s)))

# FILTER DATA

In [209]:
def filter_pacbio_flowcells(terra_table_row, cutoff_date_to_study,
                            columns_and_blacklist: dict) -> bool:
    """
    Filter applicable to all flowcells.
    :param terra_table_row:
    :param cutoff_date_to_study:
    :param columns_and_blacklist:
    :return: true if the row should be kept
    """

    # filter out known bad ones
    keep = True
    for col, black_list in columns_and_blacklist.items():
        keep &= terra_table_row[col] not in black_list

    # no time zone information
    sequencing_date = terra_table_row['created_at']
    if sequencing_date.tzinfo is None:
        return False

    # remove unknowns
    keep &= ' ' not in terra_table_row['description']
    keep &= 'unknown' != terra_table_row['description']

    # NON-genomic applications
    keep &= not terra_table_row['description'].startswith('SIRV_')
    keep &= 'amplicon' not in terra_table_row['application']

    # too early
    keep &= sequencing_date >= cutoff_date_to_study

    return keep

In [210]:
my_blacklists = {'flowcell_id': [*known_problematic_flowcells , *known_flowcells_inappropriate_for_current_pbflowcell],
                 'well_sample': known_samples_without_mercury,
                 'experiment_type': ['ISOSEQ', 'MASSEQ'],
                 'application': ['isoSeq', 'unknown']}

In [211]:
usable_flowcell_table = flowcell_table.loc[flowcell_table.apply(lambda row: filter_pacbio_flowcells(row, cutoff_date, my_blacklists), axis=1),:].reset_index(drop=True)
usable_flowcell_table.shape

(471, 69)

In [212]:
usable_flowcell_table.loc[usable_flowcell_table['flowcell_id'].isin(known_problematic_flowcells),:]

,sample,P0,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_pbi,aligned_read_length_N50,...,read_qual_mean,read_qual_median,subread_read_length_N50,subread_read_length_mean,subreads_bam,subreads_pbi,total_length,well_name,well_sample,workspace


In [213]:
print(f"{len(usable_flowcell_table['well_sample'].unique())} unique SM-[A-Z0-9]+ samples")

279 unique SM-[A-Z0-9]+ samples


In [214]:
samples_upto_date = usable_flowcell_table[['bio_sample', 'description', 'well_sample']].sort_values(by=['well_sample']).drop_duplicates(ignore_index=True)
samples_upto_date.shape

(279, 3)

In [215]:
desired_columns_in_order = ['flowcell_id', 'bio_sample', 'description', 'well_sample',
                            'aligned_est_fold_cov',
                            'lod_expected_sample',  'aligned_bam',
                            'application', 'experiment_type',
                            'is_ccs', 'is_corrected', 'is_isoseq',
                            'ccs_zmws_pass_filters_pct',
                            'instrument', 'movie_name', 'well_name', 'insert_size', 'created_at',
                            'sample', 'workspace']

# Negative LOD, i.e. swapped

In [216]:
swapped_flowcells = usable_flowcell_table[usable_flowcell_table.lod_expected_sample < -10.0].reset_index(drop=True).sort_values(by=['created_at'])
swapped_flowcells[desired_columns_in_order]

,flowcell_id,bio_sample,description,well_sample,aligned_est_fold_cov,lod_expected_sample,aligned_bam,application,experiment_type,is_ccs,is_corrected,is_isoseq,ccs_zmws_pass_filters_pct,instrument,movie_name,well_name,insert_size,created_at,sample,workspace
0,DA126105,1-06357,CG0037-9302,SM-K6JDL,9.381812,-13.977636,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,hifiReads,CCS,False,True,False,46.77,64218e,m64218e_211026_124229,A01,14000,2021-10-25 18:22:56.353000+00:00,846c0ea7-9ebf-4fe6-b9d9-2ba3e7e4be27,Gabriel_GMKFLRP_Gelb_PacBio_FY20


In [217]:
swapped_flowcells[desired_columns_in_order]\
    .to_csv(f'{output_dir}/negative.LOD.flowcells.tsv', sep='\t', header=True, index=False)

# Indecisive LOD

In [218]:
indecisive_idx = ~usable_flowcell_table['flowcell_id'].isin( borderline_lod_flowcells_cleared)
indecisive_idx &= ((usable_flowcell_table.lod_expected_sample >= -3) & (usable_flowcell_table.lod_expected_sample < 6))
indecisive_flowcells = usable_flowcell_table[indecisive_idx].reset_index(drop=True).sort_values(by=['created_at'])
indecisive_flowcells[desired_columns_in_order]

,flowcell_id,bio_sample,description,well_sample,aligned_est_fold_cov,lod_expected_sample,aligned_bam,application,experiment_type,is_ccs,is_corrected,is_isoseq,ccs_zmws_pass_filters_pct,instrument,movie_name,well_name,insert_size,created_at,sample,workspace


In [219]:
if 0 < len(indecisive_flowcells):
    indecisive_flowcells[desired_columns_in_order]\
        .to_csv(f'{output_dir}/indecisive.LOD.flowcells.tsv',
                sep='\t', header=True, index=False)

# No LOD

In [220]:
no_lod = usable_flowcell_table.loc[usable_flowcell_table.lod_expected_sample.isna(),:].reset_index(drop=True)
print(f'{len(no_lod)} flowcell have no LOD.')

6 flowcell have no LOD.


In [221]:
no_lod

,sample,P0,aligned_bai,aligned_bam,aligned_est_fold_cov,aligned_frac_bases,aligned_num_bases,aligned_num_reads,aligned_pbi,aligned_read_length_N50,...,read_qual_mean,read_qual_median,subread_read_length_N50,subread_read_length_mean,subreads_bam,subreads_pbi,total_length,well_name,well_sample,workspace
0,b371686f-d672-448a-a6aa-9400ac93f6f4,nan,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,4.843104,0.9,15013235035,1097030,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,15086,...,30.809116,31.0,15233.0,14603.0,nan,nan,104581960418,C01,SM-KTT9O,nan
1,b93737a8-ce9f-41c3-8d2b-9153f53cdeb0,nan,nan,nan,NaN,NaN,<NA>,<NA>,nan,<NA>,...,NaN,NaN,NaN,NaN,nan,nan,72538215009,C01,SM-LNN1B,nan
2,bf10cdd7-7953-4d24-9e1e-f063b44d3693,nan,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,6.127957,1.0,18996175886,1509738,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,13509,...,33.825616,35.0,13593.0,13108.0,nan,nan,61005231228,C01,SM-K6CQC,nan
3,bfcc2fd8-3844-4a95-b5e1-08bcfc9b2b3a,nan,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,5.704286,1.0,17682829870,1426884,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,13747,...,34.190260,35.0,13793.0,13061.0,nan,nan,52025549047,B01,SM-K6JCE,nan
4,dfad1eae-661f-45a6-9d1c-11097d47107b,nan,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,7.107879,0.9,22033855952,1500718,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,17194,...,32.178017,33.0,17254.0,15777.0,nan,nan,65780487854,B01,SM-LNN1A,nan
5,f2614335-e7f2-46df-85d4-9050f84dba1c,nan,nan,nan,NaN,NaN,<NA>,<NA>,nan,<NA>,...,NaN,NaN,NaN,NaN,nan,nan,70477431715,C01,SM-K6845,nan


In [222]:
is_with_bam = no_lod['aligned_bam'].apply(lambda s: s.startswith('gs://'))
is_enough_coverage = no_lod['aligned_est_fold_cov'].apply(lambda s: float(s) > 1.0)
is_with_mercury = no_lod['well_sample'].isin(samples_in_cloud_mercury)

## No LOD&mdash;meaningless coverage

In [223]:
shallow_bam = no_lod.loc[is_with_bam & ~is_enough_coverage].sort_values(by=['well_sample']).reset_index(drop=True)
shallow_bam[desired_columns_in_order]

,flowcell_id,bio_sample,description,well_sample,aligned_est_fold_cov,lod_expected_sample,aligned_bam,application,experiment_type,is_ccs,is_corrected,is_isoseq,ccs_zmws_pass_filters_pct,instrument,movie_name,well_name,insert_size,created_at,sample,workspace


In [224]:
if 0 < len(shallow_bam):
    shallow_bam.to_csv(f'{output_dir}/shallow.BAM.flowcells.tsv', sep='\t', header=True, index=False)

## No LOD&mdash;just need to run it. !!! WARN: NEED TO CHECK IT'S NOT RUNNING!!!

In [225]:
ready_to_fp = no_lod.loc[is_with_bam & is_enough_coverage & is_with_mercury].sort_values(by=['well_sample']).reset_index(drop=True)
ready_to_fp[desired_columns_in_order]

,flowcell_id,bio_sample,description,well_sample,aligned_est_fold_cov,lod_expected_sample,aligned_bam,application,experiment_type,is_ccs,is_corrected,is_isoseq,ccs_zmws_pass_filters_pct,instrument,movie_name,well_name,insert_size,created_at,sample,workspace
0,DA143913,1-04920,CG0037-9147,SM-K6CQC,6.127957,NaN,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,hifiReads,CCS,False,True,False,44.24,64020e,m64020e_211224_142457,C01,15000,2021-12-21 16:10:19.940000+00:00,bf10cdd7-7953-4d24-9e1e-f063b44d3693,nan
1,DA143445,1-05153,CG0037-9251,SM-K6JCE,5.704286,NaN,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,hifiReads,CCS,False,True,False,48.28,64297e,m64297e_211224_025130,B01,16000,2021-12-21 18:46:27.706000+00:00,bfcc2fd8-3844-4a95-b5e1-08bcfc9b2b3a,nan
2,DA143533,1-03246,CG0037-9340,SM-KTT9O,4.843104,NaN,gs://broad-gp-pacbio-outgoing/results/PBFlowce...,hifiReads,CCS,False,True,False,31.98,64271e,m64271e_211224_142908,C01,17000,2021-12-21 16:10:17.794000+00:00,b371686f-d672-448a-a6aa-9400ac93f6f4,nan


In [226]:
for flowcell in ready_to_fp['sample']:
    response = fapi.create_submission(wnamespace=primary_namespace, workspace=primary_workspace, cnamespace=primary_namespace,
                                      config='VerifyFingerprint',
                                      entity=flowcell,
                                      etype=root_data_type,
                                      use_callcache=False)  # explicitly disable call caching because database updates almost daily
    if response.ok:
        print(f"{flowcell} submitted")
    else:
        print(f"{flowcell} failed to submit due to \n  {response.json()}")

bf10cdd7-7953-4d24-9e1e-f063b44d3693 submitted
bfcc2fd8-3844-4a95-b5e1-08bcfc9b2b3a submitted
b371686f-d672-448a-a6aa-9400ac93f6f4 submitted


## No LOD&mdash;no BAM yet !!! WARN: NEED TO CHECK IT'S NOT RUNNING!!!

In [227]:
need_bam = no_lod.loc[~is_with_bam].sort_values(by=['well_sample']).reset_index(drop=True)
need_bam[desired_columns_in_order]

,flowcell_id,bio_sample,description,well_sample,aligned_est_fold_cov,lod_expected_sample,aligned_bam,application,experiment_type,is_ccs,is_corrected,is_isoseq,ccs_zmws_pass_filters_pct,instrument,movie_name,well_name,insert_size,created_at,sample,workspace
0,DA143555,1-04466,CG0037-9239,SM-K6845,NaN,NaN,nan,hifiReads,CCS,False,True,False,41.67,64297e,m64297e_211225_134822,C01,18000,2021-12-21 18:46:27.706000+00:00,f2614335-e7f2-46df-85d4-9050f84dba1c,nan
1,DA143565,1-05966,CG0038-3656,SM-LNN1B,NaN,NaN,nan,hifiReads,CCS,False,True,False,39.22,64218e,m64218e_211225_104546,C01,20000,2021-12-21 18:46:34.035000+00:00,b93737a8-ce9f-41c3-8d2b-9153f53cdeb0,nan


In [228]:
for flowcell in need_bam['sample']:
    response = fapi.create_submission(wnamespace=primary_namespace, workspace=primary_workspace, cnamespace=primary_namespace,
                                      config='PBFlowcell',
                                      entity=flowcell,
                                      etype=root_data_type)
    if response.ok:
        print(f"{flowcell} submitted")
    else:
        print(f"{flowcell} failed to submit due to \n  {response.json()}")

f2614335-e7f2-46df-85d4-9050f84dba1c submitted
b93737a8-ce9f-41c3-8d2b-9153f53cdeb0 submitted


# Query on-prem Mercury, prep for next round

In [229]:
need_mercury_sample_ids = \
    samples_upto_date.loc[~samples_upto_date['well_sample'].isin(samples_in_cloud_mercury), :]\
        .rename({'bio_sample': 'Collab_Part_ID',
                 'description': 'Collab_SM_ID',
                 'well_sample': 'Broad_LSID'
                 }, axis=1)\
        .sort_values(by=['Broad_LSID'], axis=0)\
        .reset_index(drop=True)

need_mercury_sample_ids['Broad_LSID'] = need_mercury_sample_ids['Broad_LSID'].apply(lambda s: re.sub('^SM-', '', s))

need_mercury_sample_ids['Date'] = today

print(f"{len(need_mercury_sample_ids)} newly found samples need to have their FP VCFs queried.")

2 newly found samples need to have their FP VCFs queried.


In [230]:
if 0 < len(need_mercury_sample_ids):
    csv_location = f'{output_dir}/need_mercury_sample_ids_headerless.csv'
    need_mercury_sample_ids.to_csv(csv_location,
                                   sep=',', index=False, header=False)

### !!! Now go and upload the VCFs... !!!